In [1]:
p_storage_dir = "../../storage"
p_notebooks_root = None

In [2]:
if p_notebooks_root is not None:
    import os
    os.chdir(p_notebooks_root)

In [3]:
from pathlib import Path

import numpy as np
import pandas as pd

IFG_PRODUZ_STORAGE_DIR = Path(f"{p_storage_dir}/ifg_produz")

# Formação Acadêmica

In [4]:
formacao_academica = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "extracted/formacaoacademica.csv", delimiter=";", 
                                 dtype={'ano_conclusao': 'Int64'})

formacao_academica.sample(n=5)

,id_formacao_academica,nome_instituicao,nome_curso,ano_inicio,ano_conclusao,status_curso,tipo_formacao,id_curriculo_id
4883,4874,Universidade Estadual de Londrina,Ensino de Ciências e Educação Matemática,2015,2019,CONCLUIDO,3,1776
3801,3793,Instituto Brasileiro de Educação,Pedagogia,2014,2016,CONCLUIDO,1,1371
3211,3210,Universidade Federal de Goiás,Sociologia,2013,2017,CONCLUIDO,5,1156
1457,1458,Universidade de Brasília,Biologia Animal,2010,2014,CONCLUIDO,5,514
255,256,Faculdade Jesuíta de Filosofia e Teologia,Filosofia,2008,2010,CONCLUIDO,2,87


In [5]:
formacao_academica = formacao_academica.rename(columns={
    'id_formacao_academica': 'codigo',
    'id_curriculo_id': 'codigo_curriculo'
})

## Remoção da coluna "tipo_formacao"

Essa coluna é composta apenas por números, provavelmente eram chaves estrangeiras, porém no banco de dados do IFG Produz não existe nenhuma referência que possamos descobrir o que cada número significa, então essa coluna será removida.

In [6]:
formacao_academica['tipo_formacao'].unique()

array([1, 2, 3, 5, 4])

In [7]:
formacao_academica = formacao_academica.drop(columns='tipo_formacao')

## Melhoria da ligação dos dados da coluna "nome_instituicao"

Nessa coluna temos os dados de onde foi ou está sendo realizada uma formação acadêmica, caso essa formação esteja sendo feita em alguma unidade do IFG iremos colocar a sigla da unidade em uma nova coluna, para que no momento de inserção desses dados a formação acadêmica possa ser linkada com uma unidade do IFG, melhorando a interligação dos dados do grafo.

In [8]:
import os

os.getcwd()

'/home/marcel/Repos/Personal/tcc-ifg-kg/notebooks/ifg_produz'

In [9]:
from text_processors import normalizar_string, remover_stopwords

formacao_academica['nome_instituicao_normalizado'] = formacao_academica['nome_instituicao'].apply(normalizar_string)
formacao_academica['nome_instituicao_normalizado'] = formacao_academica['nome_instituicao_normalizado'].apply(remover_stopwords)

In [10]:
formacao_academica[(formacao_academica['nome_instituicao_normalizado'].notna()) & (formacao_academica['nome_instituicao_normalizado'].str.startswith('instituto federal'))]['nome_instituicao_normalizado'].unique()

array(['instituto federal educacao ciencia tecnologia goias campus jatai',
       'instituto federal educacao ciencia tecnologia goias',
       'instituto federal goiano',
       'instituto federal goias campus uruacu',
       'instituto federal educacao ciencia tecnologia espirito santo',
       'instituto federal educacao ciencia tecnologia goias campus aparecid',
       'instituto federal educacao ciencia tecnologia',
       'instituto federal goias',
       'instituto federal educacao ciencias tecnologia goias',
       'instituto federal educacao cienicas tecnologia goias campus jatai',
       'instituto federal educacao ciencia tecnologia campus anapolis',
       'instituto federal educacao ciencia tecnologia goias campus uruacu',
       'instituto federal triangulo mineiro',
       'instituto federal goiano campus morrinhos',
       'instituto federal goiano campus urutai',
       'instituto federal parana',
       'instituto federal educacao ciencia tecnologia goias go',
       

In [11]:
formacao_academica['sigla_instituicao'] = pd.Series([None] * len(formacao_academica), dtype='object')

In [12]:
from thefuzz import fuzz

campus_to_sigla = {
    'jatai': 'JAT',
    'uruacu': 'URU',
    'aparecida': 'APA',
    'anapolis': 'ANA'        
}

def get_campus_sigla(s, threshold=90):
    for campus in campus_to_sigla.keys():
        if fuzz.partial_token_sort_ratio(campus, s) >= threshold:
            return campus_to_sigla[campus]

def get_ifg_reference(row):
    if pd.notna(row['nome_instituicao_normalizado']):
        if fuzz.token_set_ratio('instituto federal goias campus', row['nome_instituicao_normalizado']) == 100:
            campus_sigla_searched = get_campus_sigla(row['nome_instituicao_normalizado'])
            
            if campus_sigla_searched is not None:
                row['sigla_instituicao'] = campus_sigla_searched

    return row

In [13]:
formacao_academica = formacao_academica.apply(get_ifg_reference, axis=1)

In [14]:
formacao_academica[formacao_academica['sigla_instituicao'].notna()]

,codigo,nome_instituicao,nome_curso,ano_inicio,ano_conclusao,status_curso,codigo_curriculo,nome_instituicao_normalizado,sigla_instituicao
7,8,"Instituto Federal de Educação, Ciência e Tecno...",Educação para Ciências e Matemática,2012,2014,CONCLUIDO,3,instituto federal educacao ciencia tecnologia ...,JAT
170,171,INSTITUTO FEDERAL DE GOIÁS - Câmpus Uruaçu,BACHARELADO EM ENGENHARIA CIVIL,2012,2017,CONCLUIDO,57,instituto federal goias campus uruacu,URU
310,311,INSTITUTO FEDERAL DE EDUCAÇÃO CIÊNCIA E TECNOL...,CURSO LICENCIATURA EM PEDAGOGIA BILÍNGUE LIBRA...,2018,<NA>,EM_ANDAMENTO,108,instituto federal educacao ciencia tecnologia ...,APA
1252,1253,"Instituto Federal de Educação, Ciênicas e Tecn...",Programa de Pós-graduação em Educação para Ciê...,2019,<NA>,EM_ANDAMENTO,448,instituto federal educacao cienicas tecnologia...,JAT
1715,1715,"Instituto Federal de Educação, Ciência e Tecno...",Bacharelado em Engenharia Civil,2012,<NA>,EM_ANDAMENTO,604,instituto federal educacao ciencia tecnologia ...,URU
1821,1822,INSTITUTO FEDERAL DE GOIÁS - Câmpus Uruaçu,Analise e Desenvolvimento de Sistemas,2019,<NA>,EM_ANDAMENTO,643,instituto federal goias campus uruacu,URU
2803,2804,"Instituto Federal de Educação, Ciência e Tecno...",Educação para Ciências e Matemática,2013,2015,CONCLUIDO,1017,instituto federal educacao ciencia tecnologia ...,JAT
3452,3451,"Instituto Federal de Educação, Ciência e Tecno...",Educação para Ciências e Matemática,2012,2014,CONCLUIDO,1244,instituto federal educacao ciencia tecnologia ...,JAT
4039,4030,Instituto Federal de Goiás - Câmpus Jataí,Programa de Pós-Graduação em Educação para Ciê...,2018,2021,CONCLUIDO,1456,instituto federal goias campus jatai,JAT
4419,4411,"Instituto Federal de Educação, Ciência e Tecno...",Sistema da Tecnologia da Informação,2006,2010,CONCLUIDO,1603,instituto federal educacao ciencia tecnologia ...,JAT


In [15]:
formacao_academica['status_curso'].unique()

array(['CONCLUIDO', 'EM_ANDAMENTO', 'INCOMPLETO'], dtype=object)

In [16]:
list(formacao_academica.columns)

['codigo',
 'nome_instituicao',
 'nome_curso',
 'ano_inicio',
 'ano_conclusao',
 'status_curso',
 'codigo_curriculo',
 'nome_instituicao_normalizado',
 'sigla_instituicao']

In [17]:
formacao_academica.isna().any()

codigo                          False
nome_instituicao                 True
nome_curso                       True
ano_inicio                      False
ano_conclusao                    True
status_curso                    False
codigo_curriculo                False
nome_instituicao_normalizado     True
sigla_instituicao                True
dtype: bool

## Exportar

In [87]:
export_path = Path(IFG_PRODUZ_STORAGE_DIR / "preprocessed")

if not export_path.is_dir():
    Path(export_path).mkdir()

formacao_academica.to_csv(export_path / "formacao_academica.csv", sep=";", index=False)